This script performs LOO test on HLCA dataset using fleischer et al pipeline

In [10]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from tqdm import tqdm
from functools import reduce

Read HLCA data

In [11]:
hlca_smoker = sc.read_h5ad("data/step1_HLCA_smoker_processed.h5ad")
hlca_nonsmoker = sc.read_h5ad("data/step1_HLCA_nonsmoker_processed.h5ad")

Keep only these columns

In [12]:
hlca_smoker.obs = hlca_smoker.obs.loc[:,['donor_id',
                                       'age',
                                       'ann_level_1',
                                       'ann_level_2',
                                       'ann_level_3',
                                       'ann_level_4',
                                       'ann_level_5']
                                     ]
hlca_nonsmoker.obs = hlca_nonsmoker.obs.loc[:,['donor_id',
                                       'age',
                                       'ann_level_1',
                                       'ann_level_2',
                                       'ann_level_3',
                                       'ann_level_4',
                                       'ann_level_5']
                                     ]

## Modeling

In [13]:
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import mean_absolute_error

from functools import partial,reduce

Import fleischer et al prediction pipeline

In [14]:
%run fleischer_age_predictors.py

Data filtering by Number of Cells

In [15]:
'''
Filter anndata by cell type, marker genes, and subjects. Subjects are selected by min_cells
Return filtered expression matrix and ages.
'''
def filter_anndata_single_ct(anndata, ct_column, ct, donor_column, age_column, marker_genes = None, min_cells = 20):
        
    # Keep rows annotated with current cell type, and columns annotated with marker genes,  
    ct_anndata = anndata[anndata.obs[ct_column] == ct, :]
    ct_anndata = ct_anndata[:, ct_anndata.var_names.isin(marker_genes)]
    
    # Select subjects having number of cells greater than min_cells 
    subjects = ct_anndata.obs[donor_column]
    subjects_count = subjects.groupby(subjects.values).count()
    selected_subjects = subjects_count.loc[subjects_count >= min_cells].index 
    
    # Further subset anndata using the selected subjects
    ct_anndata = ct_anndata[ct_anndata.obs[donor_column].isin(selected_subjects),]
    
    # Generate filtered expression matrix and ages
    expr = ct_anndata.to_df()
    expr.index = ct_anndata.obs[donor_column].values
    ages = ct_anndata.obs[age_column]
    ages.index = expr.index
        
    return expr, ages

In [16]:
def compute_features(expr, ages, mean_degree, var_degree, n_components, use_pca, pca_model):

    if use_pca and pca_model is None:
        pca_model = PCA(n_components=n_components, whiten=True)
        expr = pd.DataFrame(pca_model.fit_transform(expr), index = expr.index)
        expr.columns = [f"PC{i+1}" for i in range(expr.shape[1])]
    elif use_pca and pca_model is not None:
        expr = pd.DataFrame(pca_model.transform(expr), index = expr.index)
        expr.columns = [f"PC{i+1}" for i in range(expr.shape[1])]
        
    # Get each subject/individual's mean expression and polynomials of mean expressions.
    subjects = expr.index.to_list()
    expr_mean = expr.groupby(subjects).mean()
    expr_mean_poly = np.hstack([expr_mean**i for i in range(1, mean_degree+1)])
    feature_names = np.hstack([[f"{col}_mean^{deg}" for col in expr.columns] for deg in range(1,mean_degree+1)]) # Name the polynomial features
    expr_mean_poly = pd.DataFrame(
                        expr_mean_poly,
                        index = expr_mean.index,
                        columns = feature_names 
                    )
    
    if var_degree > 0:

        # Get each subject/indivisual's variances.
        expr_var = expr.groupby(subjects).var()
        expr_var.fillna(value=0,inplace=True) # This should not happen 
        expr_var_poly = np.hstack([expr_var**i for i in range(1, var_degree+1)])
        feature_names = np.hstack([[f"{col}_var^{deg}" for col in expr.columns] for deg in range(1,var_degree+1)]) # Name the polynomial features
        expr_var_poly = pd.DataFrame(
                        expr_var_poly,
                        index = expr_var.index,
                        columns = feature_names 
                    )

        # Concatenate mean and var polynomial features
        X = pd.concat([expr_mean_poly,expr_var_poly],axis = 1)
    else:
        X = expr_mean_poly
    
    # Get Y
    Y = ages.groupby(subjects).mean()
    
    return X,Y,pca_model

Generate data for feature extraction and training/testing

In [17]:
import warnings
warnings.filterwarnings("ignore")

iterator = []
n_rep = 5
donor_types = ["nonsmoker","smoker"]
adatas = [hlca_nonsmoker,hlca_smoker]

for annLevel in ['ann_level_2','ann_level_3','ann_level_4','ann_level_5']:
    c1 = hlca_nonsmoker.obs[annLevel]
    c2 = hlca_smoker.obs[annLevel]
    cell_types = np.union1d(c1,c2)
    selected_cell_types = []
    
    for adata,group in zip(adatas,donor_types):
        for ct in cell_types:
            
            # Get donor number
            adata_ct = adata[adata.obs[annLevel] == ct,]
            expr,ages = filter_anndata_single_ct(adata_ct,
                                     ct_column = annLevel,
                                     ct = ct,
                                     donor_column = "donor_id",
                                     age_column = "age",
                                     marker_genes = adata.var_names,
                                     min_cells = 20
                                    )
            all_donor = expr.index.unique()
            all_donor_num = all_donor.shape[0]
            
            del(adata_ct)
            if all_donor_num >= 10 and ct != "None":
                selected_cell_types.append(ct)
                for use_pca in [False]:
                    for rep in range(1,n_rep+1):
                        iterator.append([expr,ages,group,ct,annLevel,f"all_{group}",use_pca,rep,all_donor_num])

Train and test models

In [18]:
def train_test_loo(expr, ages, use_pca, subject_type, ct):
    
    subjects = expr.index.unique()
    Y_trues = []
    Y_preds = []
    Y_trues_untrans = []
    Y_preds_untrans = []
    
    # Leave-one-out test
    for subj in subjects:
        
        test_subjects = [subj]
        train_subjects = subjects[~subjects.isin(test_subjects)]
        
        X_train = expr.loc[expr.index.isin(train_subjects),]
        Y_train = ages.loc[ages.index.isin(train_subjects),]
        X_test = expr.loc[expr.index.isin(test_subjects),]
        Y_test = ages.loc[ages.index.isin(test_subjects),]
        
        # Note that PCA trained on training data was used on test data.
        X_train, Y_train, pca_model = compute_features(X_train, Y_train, mean_degree=1, var_degree=0, n_components=10, use_pca = use_pca, pca_model=None)
        X_test, Y_test, _ = compute_features(X_test, Y_test, mean_degree=1, var_degree=0, n_components=10, use_pca=use_pca, pca_model = pca_model)
        
        # Format train and test data for model
        df_train = pd.DataFrame({"uid":Y_train.index,
              "age":Y_train.values.astype(int),
              "meta":"healthy"})
        df_train.reset_index(inplace = True, drop = True)
        X_train.reset_index(inplace = True, drop = True)
        df_train = pd.concat([df_train, X_train], axis = 1)
        df_train = df_train.set_index(['uid','age','meta']).astype(float)
        
        df_test = pd.DataFrame({"uid":Y_test.index,
              "age":Y_test.values.astype(int),
              "meta":"healthy"})
        df_test.reset_index(inplace = True, drop = True)
        X_test.reset_index(inplace = True, drop = True)
        df_test = pd.concat([df_test, X_test], axis = 1)
        df_test = df_test.set_index(['uid','age','meta']).astype(float)
        
        # Fit models
        clf = LinearDiscriminantAnalysis(shrinkage='auto',solver='eigen') 
        ensemble = subset_genes_ensemble(clf=clf, class_size=20, subset_fold=5,subset_min=0.3,dataxform_log=False,verbose=True)
        ensemble.fit(df_train, df_train.index.get_level_values('age').values)
        
        # Get predicted ages for testing set
        Y_trues.append(Y_test.values[0])
        Y_preds.append(ensemble.predict(df_test)[0])
        
    # Compute R2 and MAE as evaluation metrics
    Y_trues = np.hstack(Y_trues)
    Y_preds = np.hstack(Y_preds)
    
    r2 = r2_score(Y_trues, Y_preds)
    MAE = mean_absolute_error(Y_trues, Y_preds)
    
    return r2,MAE,Y_trues,Y_preds

### Main analysis pipeline
Adjust n_jobs based on number of cores and memory available in your machine

In [10]:
from joblib import delayed, Parallel
import warnings

def run_all(expr, ages, group, ct, ct_column, gene_type, use_pca, all_donor_num, rep):
    warnings.filterwarnings("ignore")

    # Define hyperparameter search space
    param_space = {'n_components' :0,
                 'mean_degree':1,
                 'var_degree':0
        }
    
    # Run leave-one-out test for current cell type
    r2,MAE,Y_trues,Y_preds = train_test_loo(expr, ages, use_pca, subject_type = group, ct = ct)
    #print(f"==========Test finished==========\n Ann level:{ct_column}\n Group: {group}\nCell type: {ct}\nGene type: {gene_type}\nUse pca:{use_pca}\nRep: {rep}")

    # Print leave-one-out R2 and MAE scores.
    #print(f"R2:{str(r2)}, MAE:{str(MAE)}\n")
    
    Y_trues = ",".join([str(age) for age in Y_trues])
    Y_preds = ",".join([str(age) for age in Y_preds])
    
    return ct_column,group,ct,gene_type,use_pca,r2,rep,all_donor_num,Y_trues,Y_preds

res = Parallel(n_jobs = 50)(delayed(run_all)(
                                          expr,
                                          ages,
                                          group,
                                          ct,
                                          ct_column,
                                          gene_type,
                                          use_pca,
                                          all_donor_num,
                                          rep = rep) for expr,ages,group,ct,ct_column,gene_type,use_pca,rep,all_donor_num in tqdm(iterator))

 22%|██▏       | 100/450 [02:13<09:04,  1.56s/it]

using 2605 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2520 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2584 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2595 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2477 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2592 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2533 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2541 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2603 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2435 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

using 2605 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2520 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2584 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2595 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2477 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2592 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2533 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2541 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2603 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2435 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

 33%|███▎      | 150/450 [1:33:49<4:15:32, 51.11s/it]

using 2605 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2520 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2584 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2595 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2477 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2592 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2533 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2541 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2603 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2435 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

...

using 1992 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1524 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1593 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1590 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1592 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1576 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1551 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1578 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1569 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1601 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

using 2036 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2042 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2029 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1924 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2022 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2011 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2037 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2032 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2033 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 2030 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

using 1926 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1908 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1862 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1891 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1877 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1924 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1897 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1836 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1775 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1824 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

using 3300 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3308 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3335 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3316 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3293 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3193 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3330 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3303 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3233 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 3292 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

using 1676 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1712 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1715 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1693 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1747 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1746 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1766 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1727 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1723 genes in subset requiring a max FPKM > 0.3 and > 5-fold change between max and min samples 
using 1720 genes in subset requiring a max FPKM > 0.3 and > 5-fold change

Save results

In [11]:
to_save = pd.DataFrame(res, columns = ["ann_level","group","cell_type","gene_type","use_pca","R2","rep","all_donor_num","true_age","pred_age"])

remove "[" and "]"

In [13]:
import re

In [14]:
for idx,row in to_save.iterrows():
    row["true_age"] = re.sub("\[","",row["true_age"])
    row["true_age"] = re.sub("\]","",row["true_age"])
    row["pred_age"] = re.sub("\[","",row["pred_age"])
    row["pred_age"] = re.sub("\]","",row["pred_age"])
    to_save.iloc[idx,:] = row

In [17]:
dfs = []
for grp,df in to_save.groupby(["ann_level","group","cell_type","gene_type","use_pca"]):
    df["R2_mean"] = df["R2"].mean()
    df["R2_std"] = df["R2"].std()
    del(df["R2"])
    del(df["rep"])
    dfs.append(df.drop_duplicates())
dfs = pd.concat(dfs)

to_save = to_save.sort_values(by = "R2", ascending = False)

if not os.path.isdir("results/eval"):
    os.mkdir("results/eval")    
to_save.to_csv(f"results/eval/performances_HLCA_loo_fleischer.csv",index = None)